In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import os
import loompy as lp

/data/hour/miniconda3/lib/python3.8/site-packages/numba/core/cpu.py:97: UserWarning: Numba extension module 'sparse._numba_extension' failed to load due to 'ContextualVersionConflict((numpy 1.22.4 (/data/hour/miniconda3/lib/python3.8/site-packages), Requirement.parse('numpy<1.22,>=1.18'), {'numba'}))'.
  numba.core.entrypoints.init_all()


In [3]:
# 读取含有注释信息的细胞数据
data = sc.read_h5ad('../../Sujw/GSE189357/GSE189357.raw.h5ad')

In [4]:
ann = pd.read_csv('../../Sujw/GSE189357/GSE189357_celltype.csv',index_col=0)
ann

,Major_cell_type
AAACCCAAGACCTTTG-1_TD1,T/NK cells
AAACCCAAGCCTGAAG-1_TD1,Epithelial cells
AAACCCAAGCTGTGCC-1_TD1,T/NK cells
AAACCCAAGGCCACTC-1_TD1,T/NK cells
AAACCCAAGGGTTAAT-1_TD1,T/NK cells
...,...
TTTGTTGTCAGATTGC-1_TD9,Myeloid cells
TTTGTTGTCAGCAGAG-1_TD9,Epithelial cells
TTTGTTGTCTATCCAT-1_TD9,Myeloid cells
TTTGTTGTCTGCCTCA-1_TD9,Myeloid cells


In [5]:
ann['Major_cell_type'] = ann['Major_cell_type'].replace('T/NK cells','T&NK cells')

In [5]:
data.obs = data.obs.join(ann)
data.obs

,patient,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,total_counts_hb,pct_counts_hb,Major_cell_type
AAACCCAAGACCTTTG-1_TD1,TD1,838,838,2340.0,66.0,2.820513,869.0,37.136753,0.0,0.000000,T&NK cells
AAACCCAAGCCTGAAG-1_TD1,TD1,3316,3313,6574.0,95.0,1.445087,538.0,8.183754,1.0,0.015211,Epithelial cells
AAACCCAAGCTGTGCC-1_TD1,TD1,763,763,1678.0,38.0,2.264601,464.0,27.651966,0.0,0.000000,T&NK cells
AAACCCAAGGCCACTC-1_TD1,TD1,1077,1077,3416.0,83.0,2.429742,1473.0,43.120609,0.0,0.000000,T&NK cells
AAACCCAAGGGTTAAT-1_TD1,TD1,592,592,1044.0,28.0,2.681992,243.0,23.275862,0.0,0.000000,T&NK cells
...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCAGATTGC-1_TD9,TD9,1302,1301,3060.0,164.0,5.359477,335.0,10.947713,0.0,0.000000,Myeloid cells
TTTGTTGTCAGCAGAG-1_TD9,TD9,691,691,1208.0,60.0,4.966887,191.0,15.811258,0.0,0.000000,Epithelial cells
TTTGTTGTCTATCCAT-1_TD9,TD9,316,316,526.0,25.0,4.752851,151.0,28.707224,0.0,0.000000,Myeloid cells
TTTGTTGTCTGCCTCA-1_TD9,TD9,1333,1333,3531.0,84.0,2.378929,404.0,11.441518,4.0,0.113282,Myeloid cells


In [8]:
cluster = np.unique(data.obs['Major_cell_type'])
cluster

array(['B cells', 'Endothelial cells', 'Epithelial cells', 'Fibroblasts',
       'Myeloid cells', 'T&NK cells'], dtype=object)

In [9]:
for i in cluster:
    adata = data[data.obs['Major_cell_type']==i]
    row_attrs = { 
    "Gene": np.array(adata.var.index) ,
    }
    col_attrs = { 
    "CellID":  np.array(adata.obs.index) ,
    }

    lp.create( './data/' + i + '_scenic.loom', adata.X.transpose(), row_attrs, col_attrs )